# Infer LIPID BODIES - part 8

--------------

## OBJECTIVE:  Infer sub-cellular component LIPID BODIES  in order to understand interactome 



Dependencies:
The ER  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [1]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from collections import defaultdict

import numpy as np
import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters, img_as_float
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball, disk, dilation, white_tophat, black_tophat   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot


%load_ext autoreload
%autoreload 2

if sys.platform == "win32":
    sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))
    from infer_subc.base import *
else:
    from infer_subc.base import *

viewer = None

# IMAGE PROCESSING Objective 9:  infer LIPID BODIES 
> Back to  [OUTLINE: Objective #5](#summary-of-objectives)

## summary of steps (Workflow #1 & #2)

INPUT
- ch 7
- CY mask

PRE-PROCESSING
- rescale 
-  median, window 2
-  
CORE-PROCESSING
   - "robust background" 2 STD

POST-PROCESSING
  - n/a

OUTPUT
- object BD


NOTE:  consider using [Fibrillarin](https://www.allencell.org/cell-observations/category/fibrillarin) (nucleous marker) workflow from AllenCell.  playground_spotty.ipynb  or maybe [centrin](https://www.allencell.org/cell-observations/category/centrin)





------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
##  set up files

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[5])
test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [6]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'CY_object'

# CY_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )

channel_names

['0 :: None :: Nuclei_Jan22',
 '0 :: None :: Lyso+405_Jan22',
 '0 :: None :: Mito+405_Jan22',
 '0 :: None :: Golgi+405_Jan22',
 '0 :: None :: Peroxy+405_Jan22',
 '0 :: None :: ER+405_Jan22',
 '0 :: None :: BODIPY+405low_Jan22',
 '0 :: None :: Residuals']

# WORKFLOW #1 

Not a good thing to follow from  the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.






## summary of steps


INPUT
- ch 6
- CY mask

PRE-PROCESSING
- median width = 2

CORE-PROCESSING
- global thresholding (robust background)
- 4,40 (2,40 6/28)
- outlier fraction 0.05, 0.05
- 3 standard deviations (2)
- .18,1 (.5, 1)
- intensity de-clumping
- smoothe 2

POST-PROCESSING
  - S  - remove objects less than 2x2 pixels (area = 4)

OUTPUT
- object PEROXISOMES 





In [5]:
##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets
# default_params = defaultdict(str)

default_params = DEFAULT_PARAMS


################################
# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
default_params['z_factor'] = z_factor
default_params['scale'] = scale


### INPUT

In [22]:

###################
# INPUT ER
###################
raw_lb = img_data[6,:,:,:].copy()

#LB_object, out_p =  infer_LIPID_BODY(raw_er.copy(), CY_object, out_p) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size = 4  

gaussian_smoothing_sigma = 1.3448
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(raw_lb) #  [0., 23]


mean intensity of the stack: 927.0447633531359
the standard deviation of intensity of the stack: 965.1801115030788
0.9999 percentile of the stack intensity is: 11286.563399996608
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 11.0, which is 11544.025989887004
suggested lower range is 0.5, which is 444.45470760159645
So, suggested parameter for normalization is [0.5, 11.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


### PRE-PROCESSING

In [23]:
###################
# PRE_PROCESSING
###################


intensity_norm_param = [0]  # CHECK THIS
# Linear-ish smoothing
struct_img = intensity_normalization( raw_lb ,  scaling_param=intensity_norm_param)

# edge-preserving smoothing (Option 2, used for Sec61B)
#structure_img_smooth = edge_preserving_smoothing_3d(struct_img)

# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img = median_filter_slice_by_slice( 
                                                                struct_img,
                                                                size=med_filter_size  )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [34]:
###################
# CORE_PROCESSING
###################

threshold =  get_threshold_robust_background(structure_img_smooth, num_dev = 3.5)

bw = np.zeros_like(struct_img)
bw[struct_img >= threshold] = 1

In [35]:
threshold, struct_img.mean(),struct_img.std()


(0.0444796571381049, 0.01408403789254324, 0.013679285693589229)

In [36]:
bw[:10,:10]>0

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False,  True, ..., False, False, False],
        [False, False,  True, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

### CORE PROCESSING

### POST-PROCESSING

In [39]:

###################
# POST_PROCESSING
###################
hole_max = 4 
struct_obj = aicssegmentation.core.utils.hole_filling(bw>0, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 

################################
## PARAMETERS for this step ##
minArea = 4 # 4
################################
LB_object = remove_small_objects(struct_obj, min_size=minArea, connectivity=1, in_place=False)


    # 3D cleaning


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [40]:

# viewer = napari.view_image(
#     struct_img,
#     scale=scale 
# )

viewer.add_image(
    struct_img,
    scale=scale 
)
viewer.add_image(
    raw_lb,
    scale=scale
)
viewer.add_image(
    bw,
    scale=scale
)

viewer.add_image(
    LB_object,
    scale=scale
)

<Image layer 'LB_object [2]' at 0x17cf9dfd0>

### DEFINE `infer_LIBID_BODIES` function

In [44]:
##########################
#  ### DEFINE `infer_LIBID_BODIES` function
##########################
def infer_LIBID_BODIES(struct_img, CY_object,  in_params) -> tuple:
    """
    Procedure to infer LIPID BODIES  from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the LIPID BODIES   signal

    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of LIPID BODIES  
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    intensity_norm_param = [0]  # CHECK THIS

    struct_img = intensity_normalization(struct_img, scaling_param=intensity_norm_param)
    out_p["intensity_norm_param"] = scaling_param
    # 2D smoothing
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 4   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.344
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

   ###################
    # CORE_PROCESSING
    ###################
    ################################
    ## PARAMETERS for this step ##
    num_dev = 4.0

    threshold =  get_threshold_robust_background(structure_img_smooth, num_dev = num_dev)

    struct_obj  = np.zeros_like(struct_img)
    struct_obj [struct_img >= threshold] = 1

    out_p["num_dev"] = num_dev 

    ###################
    # POST_PROCESSING
    ###################
 ###################
    # POST_PROCESSING
    ###################
    # 3D cleaning

    hole_max = 20  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 20
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max


    retval = (struct_obj,  out_p)
    return retval


# TEST  `LIPID BODIES  ` function

In [47]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'CY_object'
CY_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )


###################
# INPUT
###################
struct_img_raw = img_data[6,:,:,:].copy()

####


# DEFAULT PARAMETERS:
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

# Linear-ish smoothing
raw_lb= intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)
aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


LB_object, out_p =  infer_LIBID_BODIES(raw_lb.copy(), CY_object, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper
# possibly need to do some post-post-processing to make suer that there is only a single SO_Object?


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'LB_object'

LB_object_filen = export_ome_tiff(LB_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)


The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


intensity normalization: min-max normalization with NO absoluteintensity upper bound
mean intensity of the stack: 927.0447633531359
the standard deviation of intensity of the stack: 965.1801115030788
0.9999 percentile of the stack intensity is: 11286.563399996608
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 11.0, which is 11544.025989887004
suggested lower range is 0.5, which is 444.45470760159645
So, suggested parameter for normalization is [0.5, 11.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value
intensity normalization: min-max normalization with NO absoluteintensity upper bound
['LB_object']


The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
